In [95]:
import pandas as pd
import math

# --- Load directly from OpenFlights ---
url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"

# Correct 14-column names
colnames = [
    "id","name","city","country","iata","icao",
    "lat","lon","alt","tz","dst","tzdb","type","source"
]

airports = pd.read_csv(
    url,
    header=None,
    names=colnames,
    quotechar='"',
    skipinitialspace=True
)

# Clean IATA codes
airports['iata'] = airports['iata'].astype(str).str.strip().str.upper()
airports = airports[airports['iata'].str.match(r"^[A-Z]{3}$")]  # only real 3-letter IATA codes

# helper: get lat/lon by IATA
def coords_for(iata):
    r = airports[airports['iata'] == iata]
    if r.empty:
        raise ValueError(f"IATA {iata} not found")
    return float(r.iloc[0]['lat']), float(r.iloc[0]['lon'])

# great-circle distance (km)
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1 = math.radians(lat1); phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1); dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return R * 2 * math.asin(math.sqrt(a))

# emission estimator (simple)
DEFAULT_FACTOR_G_PER_PKM = 83.0  # g CO₂ per pax-km
def estimate_co2_kg(iata_from, iata_to, factor_g_per_pkm=DEFAULT_FACTOR_G_PER_PKM):
    lat1, lon1 = coords_for(iata_from)
    lat2, lon2 = coords_for(iata_to)
    d = haversine_km(lat1, lon1, lat2, lon2)
    grams = d * factor_g_per_pkm
    return d, grams / 1000.0

# Example use:
origin = "JFK"
dest = "LAX"
distance_km, co2_kg = estimate_co2_kg(origin, dest)
print(f"{origin}->{dest}: {distance_km:.0f} km, est {co2_kg:.1f} kg CO₂/passenger")


JFK->LAX: 3974 km, est 329.9 kg CO₂/passenger


In [103]:
import pandas as pd

# OpenFlights routes.dat URL
url_routes = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat"

# Column names for routes.dat
route_cols = [
    "airline","airline_id","source_airport","source_airport_id",
    "dest_airport","dest_airport_id","codeshare","stops","equipment"
]

# Read routes directly from OpenFlights
routes = pd.read_csv(
    url_routes,
    header=None,
    names=route_cols,
    quotechar='"',
    skipinitialspace=True
)

# Keep only routes with valid 3-letter IATA codes
routes = routes[
    (routes['source_airport'].str.match(r'^[A-Z]{3}$')) &
    (routes['dest_airport'].str.match(r'^[A-Z]{3}$'))
].reset_index(drop=True)

print("Sample routes:")
print(routes.head())
print(len(routes), "valid routes loaded")


Sample routes:
  airline airline_id source_airport source_airport_id dest_airport  \
0      2B        410            AER              2965          KZN   
1      2B        410            ASF              2966          KZN   
2      2B        410            ASF              2966          MRV   
3      2B        410            CEK              2968          KZN   
4      2B        410            CEK              2968          OVB   

  dest_airport_id codeshare  stops equipment  
0            2990       NaN      0       CR2  
1            2990       NaN      0       CR2  
2            2962       NaN      0       CR2  
3            2990       NaN      0       CR2  
4            4078       NaN      0       CR2  
67663 valid routes loaded


In [105]:
# Merge origin airport lat/lon
routes = routes.merge(
    airports[['iata', 'lat', 'lon']],
    left_on='source_airport',
    right_on='iata',
    how='left'
).rename(columns={'lat': 'source_lat', 'lon': 'source_lon'}).drop(columns='iata')

# Merge destination airport lat/lon
routes = routes.merge(
    airports[['iata', 'lat', 'lon']],
    left_on='dest_airport',
    right_on='iata',
    how='left'
).rename(columns={'lat': 'dest_lat', 'lon': 'dest_lon'}).drop(columns='iata')

# Drop any routes where lat/lon info is missing
routes = routes.dropna(subset=['source_lat', 'source_lon', 'dest_lat', 'dest_lon']).reset_index(drop=True)

print("Sample routes with coordinates:")
print(routes.head())
print(len(routes), "routes with lat/lon")


Sample routes with coordinates:
  airline airline_id source_airport source_airport_id dest_airport  \
0      2B        410            AER              2965          KZN   
1      2B        410            ASF              2966          KZN   
2      2B        410            ASF              2966          MRV   
3      2B        410            CEK              2968          KZN   
4      2B        410            CEK              2968          OVB   

  dest_airport_id codeshare  stops equipment  source_lat  source_lon  \
0            2990       NaN      0       CR2   43.449902   39.956600   
1            2990       NaN      0       CR2   46.283298   48.006302   
2            2962       NaN      0       CR2   46.283298   48.006302   
3            2990       NaN      0       CR2   55.305801   61.503300   
4            4078       NaN      0       CR2   55.305801   61.503300   

    dest_lat   dest_lon  
0  55.606201  49.278702  
1  55.606201  49.278702  
2  44.225101  43.081902  
3  55.6062

In [107]:
import math
from tqdm import tqdm  # optional, for progress bar

# Haversine distance function (already tested)
def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1 = math.radians(lat1); phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1); dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return R * 2 * math.asin(math.sqrt(a))

DEFAULT_FACTOR_G_PER_PKM = 83.0  # g CO₂ per pax-km

# Compute distance and CO₂ for each route
distances = []
co2s = []

for _, row in tqdm(routes.iterrows(), total=len(routes)):
    d = haversine_km(row['source_lat'], row['source_lon'], row['dest_lat'], row['dest_lon'])
    distances.append(d)
    co2s.append(d * DEFAULT_FACTOR_G_PER_PKM / 1000.0)  # kg CO₂

routes['distance_km'] = distances
routes['co2_kg'] = co2s

# Check
print(routes[['source_airport', 'dest_airport', 'distance_km', 'co2_kg']].head())


100%|██████████| 66934/66934 [00:01<00:00, 42987.59it/s]

  source_airport dest_airport  distance_km      co2_kg
0            AER          KZN  1506.825604  125.066525
1            ASF          KZN  1040.438320   86.356381
2            ASF          MRV   448.164909   37.197687
3            CEK          KZN   770.508500   63.952206
4            CEK          OVB  1338.631467  111.106412


In [109]:
# Filter routes for JFK -> SFO
origin = "JFK"
destination = "SFO"

jfk_sfo_routes = routes[
    (routes['source_airport'] == origin) &
    (routes['dest_airport'] == destination)
]

# Show relevant columns
print(jfk_sfo_routes[['airline', 'source_airport', 'dest_airport', 'distance_km', 'co2_kg']])


      airline source_airport dest_airport  distance_km     co2_kg
5420       AA            JFK          SFO  4151.786262  344.59826
11449      AS            JFK          SFO  4151.786262  344.59826
13873      B6            JFK          SFO  4151.786262  344.59826
20682      DL            JFK          SFO  4151.786262  344.59826
56271      UA            JFK          SFO  4151.786262  344.59826
58809      US            JFK          SFO  4151.786262  344.59826
61347      VX            JFK          SFO  4151.786262  344.59826


In [116]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# --- 1) Load BTS data ---
bts_df = pd.read_csv("airline_delay_cause.csv")

# --- 2) Inspect columns ---
print(bts_df.columns)

# --- 3) Select relevant features ---
# Adjust these based on the actual CSV column names
# Common BTS columns: 'ORIGIN', 'DEST', 'OP_CARRIER', 'CRS_DEP_TIME', 'DAY_OF_WEEK', 'MONTH', 'ARR_DELAY'
bts_df = bts_df[['ORIGIN','DEST','OP_CARRIER','CRS_DEP_TIME','DAY_OF_WEEK','MONTH','ARR_DELAY']]

# --- 4) Clean and create label ---
# Mark delayed if arrival delay > 15 minutes
bts_df['delayed'] = (bts_df['ARR_DELAY'] > 15).astype(int)

# Optional: convert departure time to hour (0-23)
bts_df['dep_hour'] = bts_df['CRS_DEP_TIME'] // 100  # e.g., 1730 -> 17

# --- 5) Model training (RandomForest) ---
features = ['dep_hour','DAY_OF_WEEK','MONTH']  # can extend with one-hot encoding for carrier, origin/dest
X = bts_df[features]
y = bts_df['delayed']

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)

# --- 6) Predict delay probability for demo ---
bts_df['delay_prob'] = model.predict_proba(X)[:,1]

print("Sample predictions:")
print(bts_df.head())


Index(['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name',
       'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')


KeyError: "None of [Index(['ORIGIN', 'DEST', 'OP_CARRIER', 'CRS_DEP_TIME', 'DAY_OF_WEEK', 'MONTH',\n       'ARR_DELAY'],\n      dtype='object')] are in the [columns]"

In [118]:
import requests
import pandas as pd

# Example BTS API endpoint for On-Time Performance
url = "https://api.transtats.bts.gov/api/1.0/airline_on_time_performance"

# API parameters
params = {
    "start_date": "2025-01-01",
    "end_date": "2025-09-30",
    "carrier": "B6",           # can be a single carrier code or "all"
    "origin": "all",
    "dest": "all",
    "format": "csv"             # BTS can return CSV
}

# Make API request
response = requests.get(url, params=params)
if response.status_code == 200:
    with open("bts_data.csv", "wb") as f:
        f.write(response.content)
    print("BTS data downloaded to bts_data.csv")
else:
    print("Error:", response.status_code, response.text)

# Load into pandas
bts_df = pd.read_csv("bts_data.csv")
print(bts_df.head())


ConnectionError: HTTPSConnectionPool(host='api.transtats.bts.gov', port=443): Max retries exceeded with url: /api/1.0/airline_on_time_performance?start_date=2025-01-01&end_date=2025-09-30&carrier=B6&origin=all&dest=all&format=csv (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000019C1A295D50>: Failed to resolve 'api.transtats.bts.gov' ([Errno 11001] getaddrinfo failed)"))

In [136]:
# Keep only relevant columns for inspection
routes_inspect = routes[['equipment']].copy()

# # Split multiple aircraft codes into lists (if needed)
# routes_inspect['equipment_list'] = routes_inspect['equipment'].str.split()

# Optional: show first 50 rows for inspection
print(routes_inspect.head(50))

# # Or just show unique equipment combinations as a dataframe
# unique_eq_df = routes_inspect[['equipment']].drop_duplicates().reset_index(drop=True)
# print(unique_eq_df)


['CR2' 'A81' 'AN4' ... '737 777 320' '330 747 777 737' 'BE1 BEH']
